In [ ]:
# Part A (25%)
# 1. Download the “dataset_lm.csv” file from Canvas and upload it to Jupyter Notebook.
# 2. Run the OLS model by using the dependent and explanatory variables in the dataset.
# 3. Show your summary table in Python and interpret your results in the summary report.
# Part B (25%)
# 1. Use error values from the OLS model to calculate their standard deviation and
# autocorrelation values for the first three lags.
# 2. Then, run the GLS model accordingly.
# 3. Show your summary table in Python and interpret your results in the summary report.
# Part C (25%)
# 1. Split the dataset into two as the training and test sets (test size = 0.5).
# 2. Run the Lasso model with alpha=1 and estimate the coefficients using the training set.
# 3. Then, calculate the mean absolute percentage error using the test set.
# 4. Find an approximate value for alpha that minimizes the mean absolute percentage error.
# Part D (25%)
# 1. Use the demand data given in the table and develop an appropriate forecasting model
# (i.e., the tailored regularization discussed in the class—see your slides for more info)
# that exploits the available information given in the table as much as possible.
# 2. Interpret your results.
